In [1]:
import pandas as pd
import requests
import datetime
api_key = '25MRYYUQTU9JVAURPS88G9E2JFFKJIBFZJ'

url = "https://api.etherscan.io/api"
from TDD import TDD

### **Functions**

In [2]:
def first_last_info(address):

    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "page": 1,
        "offset": 100,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": api_key
    }

    
    response = requests.get(url, params=params).json()
    response = response['result']
    first_date = response[0]['timeStamp']
    last_date = response[-1]['timeStamp']

    first_to = response[0]['to']
    first_from = response[0]['from']

    last_to = response[-1]['to']
    last_from = response[-1]['from']

    if first_to == address:
        first_to = 'self'
    if first_from == address:
        first_from = 'self'
    if last_to == address:
        last_to = 'self'
    if last_from == address:
        last_from = 'self'
 
    return [address,first_date,last_date,first_from,first_to,last_from,last_to]

In [3]:
def get_transaction_history(address):

    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "page": 1,
        "offset": 100,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": api_key
    }

    
    response = requests.get(url, params=params).json()
    return response['result']

def get_Erc20_transaction_history(address):

    params = {
        "module": "account",
        "action": "tokentx",
        "address": address,
        "page": 1,
        "offset": 100,
        "startblock": 0,
        "endblock": 99999999,
        "sort": "asc",
        "apikey": api_key
    }

    
    response = requests.get(url, params=params).json()
    return response['result']

def get_wallet_age(history: list[dict]):
    if len(history) > 0:
        creation_time = int(history[0]['timeStamp'])
        creation_date = datetime.datetime.fromtimestamp(creation_time).date()
        current_date = datetime.date.today()
        wallet_age = (current_date - creation_date).days
        return wallet_age
    else:
        return 0

def to_and_from(history:list[dict],address):
    from_count = 0
    to_count = 0
    for transactions in history:
        if transactions['from'] == address:
            from_count +=1
        else:
            to_count += 1
    return from_count, to_count

In [4]:
def fetch(address,nested_list):

    reg_hist = get_transaction_history(address)
    erc20_hist = get_Erc20_transaction_history(address)

    reg_age = get_wallet_age(reg_hist)
    erc_age = get_wallet_age(erc20_hist)

    reg_to,reg_from = to_and_from(reg_hist,address)
    erc_to,erc_from = to_and_from(erc20_hist,address)

    tdd,txn_count = TDD(address)

    row = [address,txn_count,reg_age,erc_age,reg_to,reg_from,erc_to,erc_from,tdd]
    nested_list.append(row)

## **Alpha ETH**

In [31]:
Alpha_Eth = pd.read_csv('voters/alpha_eth.csv')
Infra_sybils = pd.read_excel('research_data/alpha_round_sybils.xlsx', sheet_name='Eth Infra')
Alpha_Eth.dropna(subset=['supporterwallet'],inplace=True)

In [32]:
known_sybils = Infra_sybils.loc[(Infra_sybils['Suspected sybil?']==True)&(Infra_sybils['Reviewed Manually']==True)].copy()
known_sybils.rename(columns={'Source Address': 'supporterwallet'},inplace=True)

merged = pd.merge(left=known_sybils,right = Alpha_Eth,how='left',on='supporterwallet')
merged.drop(['ID', 'Token', 'Amount','Etherscan','projectid','txnhash','communityid'], axis=1, inplace=True)

In [33]:
address_counts = merged['supporterwallet'].value_counts()
Address_info_sybil = pd.DataFrame({'Address': address_counts.index, 'Count': address_counts.values})

In [34]:
funding_counts = Alpha_Eth['supporterwallet'].value_counts()
count_by_address_project = Alpha_Eth.groupby(['supporterwallet', 'project_title']).size().reset_index(name='count')
no_grants_funded = count_by_address_project['supporterwallet'].value_counts()

# create a new dataframe with the address counts as a column
Address_info1 = pd.DataFrame({'supporterwallet': funding_counts.index, 'Funding_count': funding_counts.values})
Address_info2 = pd.DataFrame({'supporterwallet': no_grants_funded.index, 'No_Grants_Funded': no_grants_funded.values})
Address_info_eth = pd.merge(Address_info1,Address_info2)

Address_info_eth.shape[0] == len(Alpha_Eth['supporterwallet'].unique())

True

In [35]:
Address_info_eth.describe()

,Funding_count,No_Grants_Funded
count,4777.000000,4777.000000
mean,4.382248,4.244714
std,5.236653,5.001276
min,1.000000,1.000000
25%,1.000000,1.000000
50%,2.000000,2.000000
75%,5.000000,5.000000
max,44.000000,22.000000


In [36]:
Address_info_eth_filtered = Address_info_eth.loc[(Address_info_eth['No_Grants_Funded']<=5)&(Address_info_eth['Funding_count']<=5)].reset_index(drop=True)

In [37]:
filtered = pd.merge(Address_info_eth_filtered,Alpha_Eth,how='left',on='supporterwallet')
filtered = filtered.drop(['projectid','txnhash','exchangerate','updatedat'],axis=1)
data_points = ['supporterwallet','Funding_count', 'No_Grants_Funded','token', 'amount', 'createdat','passportscore','passportpass']

### Querying_eth_infra

In [38]:
# headers = ['supporterwallet','txn_count','Wallet_Age','Wallet_Age(Erc20)','to_count','from_count','erc_to','erc_from','TDD']
# contents = []

# count = 0
# for i in merged['supporterwallet'].unique():
#     print(count)
#     fetch(i,contents)
#     count+=1

#sybil_alpha_sybil_data = pd.DataFrame(contents,columns=headers)
#sybil_alpha_sybil_data.to_csv('queried_data/eth_infra_sybil_data.csv',index=False)

In [39]:
# headers = ['supporterwallet','first_date','last_date','first_from','first_to','last_from','last_to']
# contents = []

# count = 0
# for i in eth_infra_q['supporterwallet']:
#     print(count)
#     info = first_last_info(i)
#     contents.append(info)
#     count+=1

# data = pd.DataFrame(contents,columns=headers)

In [40]:
# count = 0
# for i in filtered['supporterwallet'].unique():
#     print(count)
#     fetch(i,contents)
#     count+=1

# alpha_sybil_data = pd.DataFrame(contents,columns=headers)

# eth_infra_q = pd.merge(data,alpha_sybil_data,on='supporterwallet')

# alpha_sybil_data.to_csv('queried_data/eth_infra_data.csv',index=False)

In [41]:
eth_infra_q = pd.read_csv('queried_data/eth_infra_data.csv')

In [42]:
filtered['address'] = filtered['supporterwallet']

filtered['project_title_sorted'] = filtered['project_title'].apply(lambda x: '-'.join(sorted(x.lower().split())))

# group the rows by the address value and apply aggregation functions to the columns
df_result = filtered.groupby('address').agg({'supporterwallet': 'first',
                                       'project_title_sorted': '_'.join}).reset_index()

# sort the resulting DataFrame by the count of project titles in descending order
df_result = df_result.sort_values(by='project_title_sorted', ascending=False)[['supporterwallet','project_title_sorted']]

cut_filtered = filtered[data_points].drop_duplicates(subset=['supporterwallet'])

cultivate_data_infra = pd.merge(cut_filtered,df_result,on='supporterwallet',how='left')

## **Beta Round**

In [114]:
beta = pd.read_csv('voters/beta_round_votes.csv')
beta.dropna(subset=['voter'],inplace=True)

In [115]:
beta['amount'] = beta['amount'].astype(float)
beta['amount'] = beta['amount'] /10**18

In [116]:
beta

,id,transaction,blockNumber,projectId,applicationId,roundId,voter,grantAddress,token,amount,amountUSD,amountRoundToken
0,0x63b641adbdd1996d295ef79fe6c09c37dbb252133c1f...,0x0328340819a513652d554fd0f558ac2eb360b57e126e...,17123199,0x60c037f9dfa5daee0f07bc26fb73393ef924e36c73c4...,18,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0xa37d20cd446898508Ab961c6Be8d3fe1c15413fA,0x18aa467E40E1deFB1956708830A343c1D01d3D7C,0x0000000000000000000000000000000000000000,0.01000,18.211659,18223073060000000000
1,0xce4501e32c52cb08351aff61b607aab7075297a6dc7c...,0xba2f9df050c6cd43a2676eb64bc81df2e193ece3ceb7...,17123217,0xf05829c76e3da92a8942378240ab4dc70c8c509f558e...,13,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0xa37d20cd446898508Ab961c6Be8d3fe1c15413fA,0xe126b3E5d052f1F575828f61fEBA4f4f2603652a,0x0000000000000000000000000000000000000000,0.01000,18.211659,18223073060000000000
2,0xb624aed595e8ca25ce9df377d9afda1714d3a22d53b1...,0x986bdbdc0430eed93518a84dce9d98aef7385c0a9237...,17123229,0xb6a332decc38c703130a537ce7fcfe5a24bd85e52e07...,6,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0x0Cac11a7D29aB8C87D8fedB9Cc736146f4fe8E84,0x531Eb60b2dDE9a841BA358EE033533AAF90D1feA,0x0000000000000000000000000000000000000000,0.00200,3.642332,3644614600000000000
3,0xf0d07db9baf366bd868d4258db941d0b00c3026134e1...,0x986bdbdc0430eed93518a84dce9d98aef7385c0a9237...,17123229,0xbe519f8c8f7ff2acb359dd757c5d2bf25b05d9fd29b8...,9,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0x0Cac11a7D29aB8C87D8fedB9Cc736146f4fe8E84,0x3A5bd1E37b099aE3386D13947b6a90d97675e5e3,0x0000000000000000000000000000000000000000,0.00200,3.642332,3644614600000000000
4,0x866b1af3584c0bc1faf80a2c88e2b91b20be5f1529a3...,0x986bdbdc0430eed93518a84dce9d98aef7385c0a9237...,17123229,0x60c037f9dfa5daee0f07bc26fb73393ef924e36c73c4...,18,0x12BB5bBbFE596dbc489d209299B8302c3300fa40,0x0Cac11a7D29aB8C87D8fedB9Cc736146f4fe8E84,0x18aa467E40E1deFB1956708830A343c1D01d3D7C,0x0000000000000000000000000000000000000000,0.00200,3.642332,3644614600000000000
...,...,...,...,...,...,...,...,...,...,...,...,...
41502,0x41993a524ab41639de1cf91420801431cb1fd24864f9...,0xd930513e7307e7235621b553da665a72604b5067b18b...,17162196,0x68b79b3e7422bc8c33e137440f725ee0fea41df27b47...,5,0x64aa545C9C63944f8e765d9a65eda3cbbDc6e620,0xdfBecC0b4aEF80b96Da27aB483feb0892472eaC2,0x526C9C964440EBc308Ac95f3Df68feB5673A4714,0x0000000000000000000000000000000000000000,0.00070,1.319774,1320205730000000000
41503,0xfba0cd2315fb0365bf311e17a5913a642851b4d91490...,0x07e8cb36bcc090730a2955772fd56db28a6e727966a7...,17168605,0x72af4ae4148f517424e1f110574eb19cb44320fd6e98...,0,0x64aa545C9C63944f8e765d9a65eda3cbbDc6e620,0x42D47645130B5A57FE5a3d06b3Fc464D6dC9b400,0xd4AC6c14B4C96F7e66049210F56cb07468028d4e,0x0000000000000000000000000000000000000000,0.00100,1.811061,1813517430000000000
41504,0x6fcd7c528699cb53c1b2e0b045443ca46f97119a8559...,0x790fc1cbc08706648b4839c386aaace39c05e396a5b1...,17171468,0xd3bb637c6c9d44aab8806a28f8b2b7d8a1aadc0a42bb...,12,0x64aa545C9C63944f8e765d9a65eda3cbbDc6e620,0xD0352D08AA93C9bc9C79d2B3a53846DCd7c9cb92,0xD0352D08AA93C9bc9C79d2B3a53846DCd7c9cb92,0x0000000000000000000000000000000000000000,0.00893,16.328063,16336142460000000000
41505,0x1d70ca14ccf76d6d7f2844235ed178cfd88d32719b93...,0x22e4624f9a5bca9b6ef72c06f8100e5c9aab20988798...,17171468,0xd3bb637c6c9d44aab8806a28f8b2b7d8a1aadc0a42bb...,12,0x64aa545C9C63944f8e765d9a65eda3cbbDc6e620,0xD0352D08AA93C9bc9C79d2B3a53846DCd7c9cb92,0xD0352D08AA93C9bc9C79d2B3a53846DCd7c9cb92,0x0000000000000000000000000000000000000000,0.00003,0.054854,54880650000000000


In [117]:
beta.rename(columns={'voter':'supporterwallet','grantAddress':'project_addresses'},inplace=True)

In [118]:
funding_counts = beta['supporterwallet'].value_counts()
count_by_address_project = beta.groupby(['supporterwallet', 'project_addresses']).size().reset_index(name='count')
no_grants_funded = count_by_address_project['supporterwallet'].value_counts()

# create a new dataframe with the address counts as a column
Address_info_beta_1 = pd.DataFrame({'supporterwallet': funding_counts.index, 'Funding_count': funding_counts.values})
Address_info_beta_2 = pd.DataFrame({'supporterwallet': no_grants_funded.index, 'No_Grants_Funded': no_grants_funded.values})
Address_info_beta = pd.merge(Address_info_beta_1,Address_info_beta_2)

Address_info_beta.shape[0] == len(beta['supporterwallet'].unique())

True

In [119]:
Address_info_beta_filtered = Address_info_beta.loc[(Address_info_beta['No_Grants_Funded']<=10)&(Address_info_beta['Funding_count']<=10)].reset_index(drop=True)
# formerly <=5 and <10

In [120]:
filtered_beta = pd.merge(Address_info_beta_filtered,beta,how='left',on='supporterwallet')
filtered_beta = filtered_beta.drop(['id','transaction', 'blockNumber','projectId', 'applicationId', 'roundId','amountUSD','amountRoundToken'],axis=1)

In [121]:
filtered_beta

,supporterwallet,Funding_count,No_Grants_Funded,project_addresses,token,amount
0,0x504FD71b599DB92Bf7f987Ba5be049095A194F83,10,10,0xe126b3E5d052f1F575828f61fEBA4f4f2603652a,0x0000000000000000000000000000000000000000,0.00058
1,0x504FD71b599DB92Bf7f987Ba5be049095A194F83,10,10,0x99b36fDbC582D113aF36A21EBa06BFEAb7b9bE12,0x0000000000000000000000000000000000000000,0.00058
2,0x504FD71b599DB92Bf7f987Ba5be049095A194F83,10,10,0x3A5bd1E37b099aE3386D13947b6a90d97675e5e3,0x0000000000000000000000000000000000000000,0.00058
3,0x504FD71b599DB92Bf7f987Ba5be049095A194F83,10,10,0xdb0c831283C003686AEF40cA228e42bDAbf6674c,0x0000000000000000000000000000000000000000,0.00058
4,0x504FD71b599DB92Bf7f987Ba5be049095A194F83,10,10,0x02D1B4658799F61A3A627E12E17C08c59053022f,0x0000000000000000000000000000000000000000,0.00058
...,...,...,...,...,...,...
20753,0x2618055F3AD282D5F577e6e03718Cd73695DA836,1,1,0xEb439EED5642641968f9D8b52F2788e0F19B443B,0x0000000000000000000000000000000000000000,0.00100
20754,0xCF217AD74D735B020F3628308d0232e886A9FDB6,1,1,0x18aa467E40E1deFB1956708830A343c1D01d3D7C,0x0000000000000000000000000000000000000000,0.00460
20755,0x325da0a62968bd17e2D5914b2cb70A34c2c60230,1,1,0x11Ee133A1408FE2d7c62296D7eB33F234b774503,0x0000000000000000000000000000000000000000,0.00060
20756,0xf2083f557625a6b24b89Eb21265a8173d30f8F51,1,1,0x531Eb60b2dDE9a841BA358EE033533AAF90D1feA,0x0000000000000000000000000000000000000000,0.00550


In [122]:
len(filtered_beta.supporterwallet.unique())

5680

### Querying beta

In [123]:
# headers = ['supporterwallet','first_date','last_date','first_from','first_to','last_from','last_to']
# contents = []

# count = 1208
# for i in filtered_beta['supporterwallet'].unique()[1208:]:
#     print(count)
#     info = first_last_info(i)
#     contents.append(info)
#     count+=1

# data = pd.DataFrame(contents,columns=headers)

In [124]:
# headers = ['supporterwallet','txn_count','Wallet_Age','Wallet_Age(Erc20)','to_count','from_count','erc_to','erc_from','TDD']
# contents = []

# count = 1000
# for i in filtered_beta['supporterwallet'].unique()[1000:]:
#     print(count)
#     fetch(i,contents)
#     count+=1


# beta_data = pd.DataFrame(contents,columns=headers)
# df = pd.merge(data,beta_data,on='supporterwallet',how='right')
# df.to_csv('queried_data/beta_data1.csv',index=False)

In [125]:
beta_data = pd.read_csv('queried_data/beta_data.csv')

In [126]:
beta_data

,supporterwallet,first_date,last_date,first_from,first_to,last_from,last_to,txn_count,Wallet_Age,Wallet_Age(Erc20),to_count,from_count,erc_to,erc_from,TDD
0,0x73f432dA307525B6590c858bf458116322D50d16,1616995739,1627382441,0x6c03d2e5340c256838cf8b96e05e23a1a911fe0f,0x73f432da307525b6590c858bf458116322d50d16,0x73f432da307525b6590c858bf458116322d50d16,0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f,418,766,727,0,100,0,100,0.387560
1,0x8F59D415FFd78575d9C674d32aC114a41aFd3D30,1655442455,1661061630,0x8f59d415ffd78575d9c674d32ac114a41afd3d30,0xd945f759d422ae30a6166838317b937de08380e3,0x8f59d415ffd78575d9c674d32ac114a41afd3d30,0xb6728bff344e2109dbb3a0c153cad65ec4bdb736,481,321,319,0,100,0,100,0.222453
2,0xb63DF1cCb8f816426AccEaEBB6B18518b296CaD9,1629658843,1680359255,0xdfd5293d8e347dfe59e90efd55b2956a1343963d,0xb63df1ccb8f816426acceaebb6b18518b296cad9,0xb63df1ccb8f816426acceaebb6b18518b296cad9,0x2bd938cf96430b7b0879f76b010b589aeec2127c,80,620,392,0,100,0,16,4.487500
3,0xadc1d13ACf57EA12cdF14679177Fd669221ADF2c,1604104915,1634061330,0x4508f5e7d98582cd202f212090cefe542a29f9dd,0xadc1d13acf57ea12cdf14679177fd669221adf2c,0xadc1d13acf57ea12cdf14679177fd669221adf2c,0x967da4048cd07ab37855c090aaf366e4ce1b9f48,290,915,916,0,100,0,100,0.703448
4,0x3a4974C33f81bf1bBB7aA6C6Bcf1b0a616C86F30,1636936663,1669075499,0x21a31ee1afc51d94c2efccaa2092ad1028285549,0x3a4974c33f81bf1bbb7aa6c6bcf1b0a616c86f30,0x3a4974c33f81bf1bbb7aa6c6bcf1b0a616c86f30,0x495f947276749ce646f68ac8c248420045cb7b5e,230,535,551,0,100,0,84,0.369565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5685,0xe63C6E6970D80e25240f554893221Ca12D1f89E7,1665519539,1682596607,0x28ffe35688ffffd0659aee2e34778b0ae4e193ad,0xe63c6e6970d80e25240f554893221ca12d1f89e7,0xe63c6e6970d80e25240f554893221ca12d1f89e7,0x12bb5bbbfe596dbc489d209299b8302c3300fa40,46,207,149,0,50,0,14,4.086957
5686,0x0DdEf2D959eECA02A6f49765F842736435bBB71f,1613981071,1664172467,0xd551234ae421e3bcba99a0da6d736074f22192ff,0x0ddef2d959eeca02a6f49765f842736435bbb71f,0x6dea4ca70b1cdbfcb79d29920203b5ff56d31833,0x0ddef2d959eeca02a6f49765f842736435bbb71f,243,803,803,0,100,0,100,0.230453
5687,0x4ddA9E1A1Cf3a97974c31c9a7FB2Be6486332D25,1679696939,1683236291,0x21a31ee1afc51d94c2efccaa2092ad1028285549,0x4dda9e1a1cf3a97974c31c9a7fb2be6486332d25,0x4dda9e1a1cf3a97974c31c9a7fb2be6486332d25,0x4976fb03c32e5b8cfe2b6ccb31c09ba78ebaba41,35,43,42,0,44,0,9,0.914286
5688,0xA315c96c0903B6A5F0c997FDc6B257851017590B,1626665422,1632608988,0x416299aade6443e6f6e8ab67126e65a7f606eef5,0xa315c96c0903b6a5f0c997fdc6b257851017590b,0xa315c96c0903b6a5f0c997fdc6b257851017590b,0x65a58a744deb094b4af5ca7e181a9bbbf75312e2,1791,656,640,0,100,0,100,0.005025


In [127]:
beta_points =['supporterwallet','Funding_count','No_Grants_Funded']

In [130]:
filtered_beta['address'] = filtered_beta['supporterwallet']

filtered_beta['project_title_sorted'] = filtered_beta['project_addresses'].apply(lambda x: '-'.join(sorted(x.lower().split())))

# group the rows by the address value and apply aggregation functions to the columns
df_result = filtered_beta.groupby('address').agg({'supporterwallet': 'first',
                                       'project_title_sorted': '_'.join}).reset_index()

# sort the resulting DataFrame by the count of project titles in descending order
df_result = df_result.sort_values(by='project_title_sorted', ascending=False)[['supporterwallet','project_title_sorted']]

cut_filtered_beta = filtered_beta[beta_points].drop_duplicates(subset=['supporterwallet'])

cultivate_data_beta = pd.merge(cut_filtered_beta,df_result,on='supporterwallet',how='left')

## **Grouping**

In [132]:
stats_data_infra = pd.merge(cultivate_data_infra,eth_infra_q,on='supporterwallet')
stats_data_beta = pd.merge(cultivate_data_beta,beta_data,on='supporterwallet')
stats_data_beta.drop_duplicates(subset='supporterwallet',inplace=True)

In [133]:
stats_data_infra = stats_data_infra.reindex(sorted(stats_data_infra.columns), axis=1)
stats_data_beta = stats_data_beta.reindex(sorted(stats_data_beta.columns), axis=1)

In [134]:
stats_data_beta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5680 entries, 0 to 5689
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Funding_count         5680 non-null   int64  
 1   No_Grants_Funded      5680 non-null   int64  
 2   TDD                   5680 non-null   float64
 3   Wallet_Age            5680 non-null   int64  
 4   Wallet_Age(Erc20)     5680 non-null   int64  
 5   erc_from              5680 non-null   int64  
 6   erc_to                5680 non-null   int64  
 7   first_date            5680 non-null   int64  
 8   first_from            5680 non-null   object 
 9   first_to              5680 non-null   object 
 10  from_count            5680 non-null   int64  
 11  last_date             5680 non-null   int64  
 12  last_from             5680 non-null   object 
 13  last_to               5680 non-null   object 
 14  project_title_sorted  5680 non-null   object 
 15  supporterwallet      

In [135]:
stats_data_beta.fillna(0.0,inplace=True)
stats_data_infra.fillna(0.0,inplace=True)

In [136]:
stats_data_i = stats_data_infra.copy()
stats_data_b = stats_data_beta.copy()


In [137]:
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ['token', 'project_title_sorted', 'first_from', 'first_to', 'last_from', 'last_to']

for col in columns_to_encode:
    le = LabelEncoder()
    stats_data_i[col] = le.fit_transform(stats_data_i[col])

columns_to_encode = ['project_title_sorted', 'first_from', 'first_to', 'last_from', 'last_to']

for col in columns_to_encode:
    le = LabelEncoder()
    stats_data_b[col] = le.fit_transform(stats_data_b[col])
    print(col)

project_title_sorted
first_from
first_to
last_from
last_to


In [138]:
feature_infra = ['Funding_count', 'No_Grants_Funded', 'token','amount', 'passportscore', 'passportpass',
            'project_title_sorted','first_from','first_to', 'last_from', 'last_to', 'txn_count', 'Wallet_Age',
            'Wallet_Age(Erc20)', 'to_count', 'from_count', 'erc_to', 'erc_from']

###  Eth Infra

In [139]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

data = np.array(stats_data_i[feature_infra])

# Add supporterwallet column to data
supporterwallet_col = stats_data_i['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.9999

# Initialize list to store similar supporter wallets
similar_supporters_infra = []

# Loop through similarity matrix and group together similar supporter wallets
for i in range(len(similarity_matrix)):
    similar_supporter_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_supporter_indices) > 1:
        similar_supporter_wallets = [data[j][0] for j in similar_supporter_indices]
        for supporter in similar_supporter_wallets:
            if supporter not in similar_supporters_infra:
                similar_supporters_infra.append(supporter)

In [140]:
data = stats_data_i[feature_infra].astype(float)

# Add supporterwallet column to data
supporterwallet_col = stats_data_i['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.9999

# Initialize list to store similar rows
similar_rows = []

# Loop through similarity matrix and group together similar rows
for i in range(len(similarity_matrix)):
    similar_row_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_row_indices) > 1:
        similar_row_values = [tuple(data[j]) for j in similar_row_indices]
        if similar_row_values not in similar_rows:
            similar_rows.append(similar_row_values)

# Print out the similar rows
for i, row_group in enumerate(similar_rows):
    print(f"Similar Row Group {i}:")
    for row in row_group:
        print(row[0], row[1:])
    print()


Similar Row Group 0:
0x992129ad0e5c642f4d088ba6b51da823588b09e1 (4.0, 4.0, 1.0, 0.0019, 22.24, 1.0, 310.0, 1221.0, 90.0, 151.0, 493.0, 29.0, 160.0, 158.0, 29.0, 8.0, 1.0, 2.0)
0x8be31a599d12b6bf92a21adc211b0065505915af (4.0, 4.0, 1.0, 0.0019407, 22.22, 1.0, 310.0, 1221.0, 90.0, 151.0, 493.0, 33.0, 160.0, 158.0, 33.0, 9.0, 1.0, 2.0)

Similar Row Group 1:
0xfbbd5070addc38f53f95bc5259c2d0f9e90a92eb (3.0, 3.0, 1.0, 0.00065, 35.69, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 46.0, 675.0, 704.0, 46.0, 26.0, 3.0, 5.0)
0xfa6b1a941c8f9515ab2122b47b4c806efb13b7b9 (3.0, 3.0, 1.0, 0.00065, 35.69, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 50.0, 675.0, 704.0, 50.0, 26.0, 3.0, 6.0)
0xa879b4fef26dfd1cd35843ddd6071f2bc63ebfb7 (3.0, 3.0, 1.0, 0.00065, 35.69, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 45.0, 675.0, 704.0, 45.0, 25.0, 3.0, 6.0)
0xb03ae245b2bb3526fdb511954b3775f11db12458 (3.0, 3.0, 1.0, 0.00065, 30.71, 1.0, 437.0, 1382.0, 43.0, 149.0, 1260.0, 44.0, 675.0, 704.0, 44.0, 26.0, 3.0, 7.0)

Similar Row

In [141]:
similar_supporters_infra

['0x992129ad0e5c642f4d088ba6b51da823588b09e1',
 '0x8be31a599d12b6bf92a21adc211b0065505915af',
 '0xfbbd5070addc38f53f95bc5259c2d0f9e90a92eb',
 '0xfa6b1a941c8f9515ab2122b47b4c806efb13b7b9',
 '0xa879b4fef26dfd1cd35843ddd6071f2bc63ebfb7',
 '0xb03ae245b2bb3526fdb511954b3775f11db12458',
 '0xfdee0c33f3493a1b83cc63bb5fbfa766572fbc03',
 '0xc900edd003feea41c0efb57262ba3d70608e7e22',
 '0xfc888fadd4063cb98041a59a232183a6d4306da2',
 '0xe6a1ff1cdb17130212e8a1ff596b8beadf112735',
 '0xcbc0012dd50f9e3920d55ea25aa6a22104c1c9ca',
 '0x90b5c0a0f414c7824d54802c5ab0c9fb10fb82d8',
 '0x508d4589ba566f94edb89a4ab15466ee128dffc3',
 '0x7e646d9f5ee2947c66e87aa414f8a5e7b26dd380',
 '0x5f0eb11a1232a771fcb91f7f796a5ee4cb217217',
 '0x68c882487920d8c114b9f80c4bbf882f60befc5b',
 '0xa2b1c8798cac94cab5a7b493a473979a724e1a1c',
 '0x2e884770f9e11ba6b649039e34f7504ada6ad8bb',
 '0x29d6f812e973dcbbc2e28d3c24c4baf371afd893',
 '0xc6d0b227ffcb0c7bb7eef6cde19e173a6b3ea7d5',
 '0xbdbe9e680d06fa764976d7ef2eee8cece745051d',
 '0x4e088ee46

In [142]:
infra_identified_sybils = pd.DataFrame(similar_supporters_infra,columns=['address'])
infra_identified_sybils.to_csv('identified_sybils/eth_infra.csv',index=False)

### Beta

In [143]:
feature_beta = ['Funding_count', 'No_Grants_Funded', 'TDD', 'Wallet_Age',
       'Wallet_Age(Erc20)', 'erc_from', 'erc_to',
       'first_to', 'from_count',
       'project_title_sorted', 'to_count', 'txn_count']

In [144]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

data = np.array(stats_data_b[feature_beta])

# Add supporterwallet column to data
supporterwallet_col = stats_data_b['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.999999

# Initialize list to store similar supporter wallets
similar_supporters_beta = []

# Loop through similarity matrix and group together similar supporter wallets
for i in range(len(similarity_matrix)):
    similar_supporter_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_supporter_indices) > 1:
        similar_supporter_wallets = [data[j][0] for j in similar_supporter_indices]
        for supporter in similar_supporter_wallets:
            if supporter not in similar_supporters_infra:
                similar_supporters_beta.append(supporter)

In [145]:
similar_supporters_beta

['0xe47076831cB8C595AB763A7bc94c8b1fb4b7f767',
 '0xe3f1C7f28b39f136e2e8B5a6D04d0C1B3eCD4a03',
 '0xe47076831cB8C595AB763A7bc94c8b1fb4b7f767',
 '0xe3f1C7f28b39f136e2e8B5a6D04d0C1B3eCD4a03',
 '0x81cc36DdA894256aa95458F78B4029381b09BDfb',
 '0x8124eFC94c951cF41D4B0B42794C678458a00726',
 '0xD71dD4B76E2973f34c73c15Bc0b05Cf919D7e4A3',
 '0xD57534A54Bd7a31E446FCE08E2CcbB0F7310F526',
 '0xD2E5017b43119fE4b84C2C3e3076Ca072f3D74c6',
 '0x81cc36DdA894256aa95458F78B4029381b09BDfb',
 '0x8124eFC94c951cF41D4B0B42794C678458a00726',
 '0x7F6221B000089d6f2Dd9e0D31a81CE79821f2fA0',
 '0x7F91F0CAa692105D8010387ACF3660900912C78f',
 '0xD6a09E547EEF94985ac86aB53A51E5c511F42D40',
 '0xD69797db77b2A819295eaF6A2E0e330c434e45b7',
 '0xD6a09E547EEF94985ac86aB53A51E5c511F42D40',
 '0xD69797db77b2A819295eaF6A2E0e330c434e45b7',
 '0xD71dD4B76E2973f34c73c15Bc0b05Cf919D7e4A3',
 '0xD57534A54Bd7a31E446FCE08E2CcbB0F7310F526',
 '0xD2E5017b43119fE4b84C2C3e3076Ca072f3D74c6',
 '0x7F6221B000089d6f2Dd9e0D31a81CE79821f2fA0',
 '0x7F91F0CAa

In [148]:
data = stats_data_b[feature_beta]

# Add supporterwallet column to data
supporterwallet_col = stats_data_b['supporterwallet'].values.reshape(-1, 1)
data = np.hstack((supporterwallet_col, data))

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(data[:, 1:])

# Set threshold for grouping together similar rows
threshold = 0.9999999

# Initialize list to store similar rows
similar_rows = []

# Loop through similarity matrix and group together similar rows
for i in range(len(similarity_matrix)):
    similar_row_indices = np.where(similarity_matrix[i] >= threshold)[0]
    if len(similar_row_indices) > 1:
        similar_row_values = [tuple(data[j]) for j in similar_row_indices]
        if similar_row_values not in similar_rows:
            similar_rows.append(similar_row_values)

# Print out the similar rows
for i, row_group in enumerate(similar_rows):
    print(f"Similar Row Group {i}:")
    for row in row_group:
        print(row[0], row[1:])
    print()


Similar Row Group 0:
0x633D4Cd6bB4d0c227792537Ad006705A4191cE02 (4.0, 4.0, 6.619047619047619, 150.0, 71.0, 5.0, 0.0, 199.0, 25.0, 1018.0, 0.0, 21.0)
0xb62462465E2697F2275F668E775710Fa24fC9394 (4.0, 4.0, 6.619047619047619, 150.0, 71.0, 5.0, 0.0, 199.0, 25.0, 1018.0, 0.0, 21.0)

Similar Row Group 1:
0x9234938dB2d9bD6D944d27f4F7b17187c81a5d7A (4.0, 4.0, 7.666666666666667, 150.0, 71.0, 5.0, 0.0, 199.0, 21.0, 1018.0, 0.0, 18.0)
0x7BE839eA6214F231f0Bb893EE6dC6b62506f7f6c (4.0, 4.0, 7.666666666666667, 150.0, 71.0, 5.0, 0.0, 199.0, 21.0, 1018.0, 0.0, 18.0)

Similar Row Group 2:
0x33d7b053E9fF523E4dA6be3108CC9D67A0d610E8 (4.0, 4.0, 8.625, 150.0, 71.0, 5.0, 0.0, 199.0, 18.0, 1018.0, 0.0, 16.0)
0x300C018D36d029Bf1C6a36280A7da22d64968ec4 (4.0, 4.0, 8.625, 150.0, 71.0, 5.0, 0.0, 199.0, 18.0, 1018.0, 0.0, 16.0)

Similar Row Group 3:
0xB33F3Cbf89ca501CdD903bFcEFeee6B66d7A6C42 (4.0, 4.0, 8.117647058823529, 150.0, 71.0, 5.0, 0.0, 199.0, 19.0, 525.0, 0.0, 17.0)
0xb8ecE7d8E375056a1429d9109DaeF89384F7387a

In [149]:
beta_identified_sybils = pd.DataFrame(similar_supporters_beta,columns=['address'])
beta_identified_sybils.to_csv('identified_sybils/beta.csv',index=False)